Sources:

https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

https://arxiv.org/pdf/1603.00748.pdf

https://github.com/BY571/Normalized-Advantage-Function-NAF-/blob/master/NAF.ipynb

https://towardsdatascience.com/applied-reinforcement-learning-v-normalized-advantage-function-naf-for-continuous-control-62ad143d3095

In [ ]:
import gymnasium as gym
import numpy as np
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random
import copy

In [ ]:
env = gym.make('MountainCarContinuous-v0')

In [ ]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# define the transition
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

# define the replay memory
class ReplayMemory(object):
    """Implements a Replay Memory."""

    def __init__(self, capacity):
        """Initialize a ReplayMemory object."""
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        """Sample a batch_size of transitions from the memory."""
        return random.sample(self.memory, batch_size)

    def __len__(self):
        """Return the length of the memory."""
        return len(self.memory)



In [ ]:

class DQN(nn.Module):
    """The Q-function approximation via a Deep Neural Network."""

    def __init__(self, observation_size, action_size):
        """Initialize parameters and build model."""
        super().__init__()  

        self.layer1 = nn.Linear(observation_size, 128)
        self.bn1 = nn.BatchNorm1d(128) # batch normalization, see https://arxiv.org/abs/1502.03167, 
        self.layer2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128) # batch normalization

        self.V = nn.Linear(128, 1) # value function
        self.mu = nn.Linear(128, 1) # mean of the action distribution
        self.L = nn.Linear(128, 1) # lower triangular matrix of the cholesky decomposition of the covariance matrix, here just a single value

        

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x, action=None):
        

        # propagate the input state
        x = F.relu(self.layer1(x))
        # x = self.bn1(x)
        x = F.relu(self.layer2(x))
        # x = self.bn2(x)

        # get the estimate of the value of state x
        V = self.V(x) 

        # get the mu parameter of the advantage function
        mu = torch.tanh(self.mu(x)) 

        # get the P parameer of the advantage function
        L = torch.tanh(self.L(x))
        P = L * L

        Q = None
        if action is not None:

            # assemble the advantage function
            A = -0.5 * (action - mu) * (action - mu) * P 

            # calculate the q value for the state action pair
            Q = V + A

        return Q, V, mu

In [ ]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
# LR = 1e-3 # commented so that the parameter can be controlled from outside the notebook

# Get number of actions from gym action space
action_size = 1

# Get the number of state observations
state, info = env.reset()
observation_size = len(state)

# Initialize the policy and target networks
policy_net = DQN(observation_size, action_size).to(device)
target_net = DQN(observation_size, action_size).to(device)
target_net.load_state_dict(policy_net.state_dict())  # copy weights

# Initialize the optimizer
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True) 

# Initialize the replay memory
memory = ReplayMemory(10000)

# Initialize the noise process
seed = random.seed(42)
noise = OUNoise(1, seed, theta=0.15, sigma=0.2) 


steps_done = 0


def select_action(state):
    """Select an action from the policy network with added noise from the noise process."""
    global steps_done

    steps_done += 1
    with torch.no_grad():

        # NEW: added .view(1, 1)
        return policy_net(state)[2].view(1, 1) + noise.sample() # QUESTION: what does view do?

# track the duration of each episode aand the scores
episode_durations = []
episode_scores = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())


In [ ]:
def plot_scores(show_result=False):
    plt.figure(1)
    scores_t = torch.tensor(episode_scores, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.plot(scores_t.numpy())
    # Take 100 episode averages and plot them too
    if len(scores_t) >= 100:
        means = scores_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())



In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    
    # sample transitions
    transitions = memory.sample(BATCH_SIZE)
    
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    
    state_action_values = policy_net(state_batch, action_batch)[0]

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states)[1].reshape(-1)
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch 

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()



In [ ]:
# set up diagnostics
naf_diagnostics = {'rewards': [],
               'actions': []}

In [ ]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 400

for i_episode in range(num_episodes):

    actions = []
    rewards = []

    # Initialize the environment and get it's state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        actions.append(action)
        rewards.append(reward)

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

    

        if done:
            episode_durations.append(t + 1)
            episode_scores.append(sum(rewards))
            plot_scores()
            noise.reset()
            break
    noise.reset()

    naf_diagnostics['rewards'].append(rewards)
    naf_diagnostics['actions'].append(actions)


print('Complete')
plot_scores(show_result=True)
plt.ioff()
plt.show()